# Make the final SXDS catalogue

We want to take all the columns. We will save a catalogue for each patch which can be stacked to produce the final catalogues for serving publicly. The is_inner flags are used to remove the overlap regions. As far as possible we try to mimic the tables produced for HSC PDR2.

In [1]:
import os
import json
import numpy as np
import lsst.daf.persistence as dafPersist

from astropy.table import Table, join, vstack
import astropy.units as u

import warnings
warnings.filterwarnings("ignore")

In [2]:
hscBands = ['G', 'R', 'I', 'Z', 'Y']
vistaBands = ['Z', 'Y', 'J', 'H', 'Ks']
allBands = ['HSC-' +b for b in hscBands] + ['VISTA-' +b for b in vistaBands]

In [3]:
allBands

['HSC-G',
 'HSC-R',
 'HSC-I',
 'HSC-Z',
 'HSC-Y',
 'VISTA-Z',
 'VISTA-Y',
 'VISTA-J',
 'VISTA-H',
 'VISTA-Ks']

In [4]:
hscBands

['G', 'R', 'I', 'Z', 'Y']

In [5]:
BUTLER_LOC = '/Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data'
#BUTLER_LOC = '../../dmu4/dmu4_SXDS/data'
DATA = '/Volumes/Raph500/lsst-ir-fusion/dmu5/dmu5_Example/data'
#DATA =  './data'
butler =  dafPersist.Butler(inputs='{}/rerun/coaddForcedPhot'.format(BUTLER_LOC))

In [6]:
full_patch_dict = json.load(open('../../dmu4/dmu4_SXDS/full_patch_dict.json', 'r'))

In [7]:
full_patch_dict

{'8282': ['0,8'],
 '8283': ['0,8',
  '1,7',
  '1,8',
  '2,7',
  '2,8',
  '3,7',
  '3,8',
  '4,7',
  '4,8',
  '5,7',
  '5,8',
  '6,7',
  '6,8',
  '7,7',
  '7,8',
  '8,8'],
 '8284': ['8,8'],
 '8523': ['0,1',
  '0,2',
  '0,3',
  '0,4',
  '0,5',
  '0,6',
  '0,7',
  '0,8',
  '1,1',
  '1,2',
  '1,3',
  '1,4',
  '1,5',
  '1,6',
  '1,7',
  '1,8',
  '2,1',
  '2,2',
  '2,3',
  '2,4',
  '2,5',
  '2,6',
  '2,7',
  '2,8',
  '3,1',
  '3,2',
  '3,3',
  '3,4',
  '3,5',
  '3,6',
  '3,7',
  '3,8',
  '4,1',
  '4,2',
  '4,3',
  '4,4',
  '4,5',
  '4,6',
  '4,7',
  '4,8',
  '5,2',
  '5,3',
  '5,4',
  '5,5',
  '5,6',
  '5,7',
  '5,8',
  '6,3',
  '6,4',
  '6,5',
  '6,6',
  '6,7',
  '6,8',
  '7,3',
  '7,4',
  '7,5',
  '7,6',
  '7,7',
  '7,8'],
 '8524': ['0,0',
  '0,1',
  '0,2',
  '0,3',
  '0,4',
  '0,5',
  '0,6',
  '0,7',
  '0,8',
  '1,0',
  '1,1',
  '1,2',
  '1,3',
  '1,4',
  '1,5',
  '1,6',
  '1,7',
  '1,8',
  '2,0',
  '2,1',
  '2,2',
  '2,3',
  '2,4',
  '2,5',
  '2,6',
  '2,7',
  '2,8',
  '3,0',
  '3,1',
  

In [8]:
def addFlux(cat, sources, photoCalib):
    """Add magnitudes and fluxes to an astropy catalogues with instrument fluxes"""
    for c in cat.colnames:
        if (c.endswith('_instFlux')):
            try:
                mags = photoCalib.instFluxToMagnitude(sources, c.replace('_instFlux',''))
                flux = photoCalib.instFluxToNanojansky(sources, c.replace('_instFlux',''))
                cat["{}_mag".format(c.replace('_instFlux',''))] = mags[:,0]
                cat["{}_mag".format(c.replace('_instFlux',''))].unit = u.mag
                cat["{}_mag".format(c.replace('_instFlux',''))].unit = cat[c].description.replace('instFlux', 'mag')
                
                cat["{}_magErr".format(c.replace('_instFlux',''))] = mags[:,1]
                cat["{}_magErr".format(c.replace('_instFlux',''))].unit = u.mag
                cat["{}_magErr".format(c.replace('_instFlux',''))].unit = cat[c].description.replace('instFlux', 'mag')
                
                cat["{}_flux".format(c.replace('_instFlux',''))] = flux[:,0]
                cat["{}_flux".format(c.replace('_instFlux',''))].unit = u.nJy
                cat["{}_flux".format(c.replace('_instFlux',''))].unit = cat[c].description.replace('instFlux', 'flux')
                
                cat["{}_fluxErr".format(c.replace('_instFlux',''))] = flux[:,1]
                cat["{}_fluxErr".format(c.replace('_instFlux',''))].unit = u.nJy
                cat["{}_fluxErr".format(c.replace('_instFlux',''))].unit = cat[c].description.replace('instFlux', 'flux')
            except:
                pass
    return cat

def makeCat(tract, patch, BUTLER_LOC):
    """make the final catalogue on a given patch for later stacking"""
    cat =Table()
    tract = int(tract)
    for band in allBands:
        
        try:
            CoaddCalexp = butler.get('deepCoadd_calexp',  {'filter': band, 'tract': tract, 'patch': patch})
            CoaddPhotoCalib = CoaddCalexp.getPhotoCalib()
        
            measSources = butler.get('deepCoadd_meas', {'filter': band, 'tract': tract, 'patch': patch})
            measCat = measSources.asAstropy()
            measCat = addFlux(measCat, measSources, CoaddPhotoCalib)
            for c in measCat.colnames:    
                if c != 'id':
                    measCat[c].name = "{}_{}_{}".format(band,'m', c).replace('oSecondDerivative','oSD')
                
            forcedSources = butler.get('deepCoadd_forced_src', {'filter': band, 'tract': tract, 'patch': patch})
            forcedCat = forcedSources.asAstropy()
            forcedCat = addFlux(forcedCat, forcedSources, CoaddPhotoCalib)
            for c in forcedCat.colnames:    
                if c != 'id':
                    forcedCat[c].name = "{}_{}_{}".format(band,'f', c).replace('oSecondDerivative','oSD')
                    
            if len(cat)==0:
                #On first band no join
                cat = measCat
            else:
                #After first band join tables in
                cat = join(cat, measCat,join_type='outer')
            cat = join(cat, forcedCat,join_type='outer')
        except:
            warnings.warn("Band {} failed.".format(band))
                

    if len(cat) == 0:
        cat=None
    return cat

#makeCat(8524, '3,3', BUTLER_LOC)

In [9]:
cat = makeCat(8524, '3,3', BUTLER_LOC)
cat[cat.colnames[0:999]].write(DATA+'/full_cat_{}_{}.fits'.format(8524, '3,3'), overwrite=True)
print('success', len(cat.colnames))

DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-G/8524/3,3/meas-HSC-G-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/HSC-G/8524/3,3/forcedSrc-HSC-G-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-R/8524/3,3/meas-HSC-R-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/HSC-R/8524/3,3/forcedSrc-HSC-R-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-I/8524/3,3/meas-HSC-I-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/HSC-I/8524/3,3/forcedSrc-HSC-I-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-Z/8524/3,3/meas-HSC-Z-8524-3,3.fits
DEBUG:  

In [10]:
cat.colnames[0:5]

['id',
 'HSC-G_m_coord_ra',
 'HSC-G_m_coord_dec',
 'HSC-G_m_parent',
 'HSC-G_m_merge_footprint_HSC-G']

In [11]:
cat['HSC-G_m_slot_CalibFlux_instFlux'].description

'instFlux within 12.000000-pixel aperture'

In [12]:
cols = Table()
cols['name'] = cat.colnames
cols['description'] = [cat[c].description for c in cat.colnames]
cols['unit'] = [str(cat[c].unit) for c in cat.colnames]
cols['type'] = [cat[c].dtype for c in cat.colnames]
cols.write('./columns_descriptions.csv')

In [13]:
reduced_cols = [ 'id', 'VISTA-Ks_m_coord_ra', 'VISTA-Ks_m_coord_dec']
reduced_cols += ['{}_m_base_CircularApertureFlux_12_0_mag'.format(b) for b in allBands]
reduced_cols += ['{}_m_base_CircularApertureFlux_12_0_magErr'.format(b) for b in allBands]
reduced_cols += ['{}_m_base_CircularApertureFlux_12_0_flag'.format(b) for b in allBands]
reduced_cols

['id',
 'VISTA-Ks_m_coord_ra',
 'VISTA-Ks_m_coord_dec',
 'HSC-G_m_base_CircularApertureFlux_12_0_mag',
 'HSC-R_m_base_CircularApertureFlux_12_0_mag',
 'HSC-I_m_base_CircularApertureFlux_12_0_mag',
 'HSC-Z_m_base_CircularApertureFlux_12_0_mag',
 'HSC-Y_m_base_CircularApertureFlux_12_0_mag',
 'VISTA-Z_m_base_CircularApertureFlux_12_0_mag',
 'VISTA-Y_m_base_CircularApertureFlux_12_0_mag',
 'VISTA-J_m_base_CircularApertureFlux_12_0_mag',
 'VISTA-H_m_base_CircularApertureFlux_12_0_mag',
 'VISTA-Ks_m_base_CircularApertureFlux_12_0_mag',
 'HSC-G_m_base_CircularApertureFlux_12_0_magErr',
 'HSC-R_m_base_CircularApertureFlux_12_0_magErr',
 'HSC-I_m_base_CircularApertureFlux_12_0_magErr',
 'HSC-Z_m_base_CircularApertureFlux_12_0_magErr',
 'HSC-Y_m_base_CircularApertureFlux_12_0_magErr',
 'VISTA-Z_m_base_CircularApertureFlux_12_0_magErr',
 'VISTA-Y_m_base_CircularApertureFlux_12_0_magErr',
 'VISTA-J_m_base_CircularApertureFlux_12_0_magErr',
 'VISTA-H_m_base_CircularApertureFlux_12_0_magErr',
 'VIS

In [17]:
name_base = DATA+'/full_cat_{}_{}_tab{}.fits'
reduced_cat = Table()
for tract in full_patch_dict:
    for patch in full_patch_dict[tract]:
        #print(type(tract), type(patch))
        #skip if file exists
        if os.path.exists(DATA+'/full_cat_{}_{}_tab5.fits'.format(tract, patch)):
            print('Tract {}, patch {} already done'.format(tract, patch))
            continue
        try:
            cat = makeCat(tract, patch, BUTLER_LOC)
            print('pass2')
            bounds = np.append(np.arange(0,len(cat.colnames),999),len(cat.colnames))
            for n in np.arange(len(bounds)-1):
                #Write tables in 1000 columns blocks
                #print(bounds[n], bounds[n+1])
                cat[cat.colnames[bounds[n]:bounds[n+1]]].write(name_base.format(
                    tract,patch,n
                ), overwrite=True)
                #print('written')
            mask = cat['VISTA-Ks_m_detect_isPatchInner'] & cat['VISTA-Ks_m_detect_isTractInner']
            reduced_cat = vstack([reduced_cat, cat[mask][reduced_cols]])
            print('success')
        except:
            pass #print('Failed.')

pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
Tract 8524, patch 3,3 already done
Tract 8524, patch 3,4 already done
Tract 8524, patch 3,5 already done
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-G/8524/3,6/meas-HSC-G-8524-3,6.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-Z/8524/3,6/meas

pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2
pass2


In [ ]:
len(reduced_cat)

In [15]:
reduced_cat[:5]

AttributeError: 'NoneType' object has no attribute 'colnames'

In [ ]:
reduced_cat.write(DATA+'redcued_cat.fits', overwrite=True)